In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from tf_keras.preprocessing.image import ImageDataGenerator
import tf_keras
from tf_keras.applications import VGG16
from tf_keras import layers, models

2024-08-28 22:39:35.743109: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-28 22:39:35.812249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 22:39:35.841669: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 22:39:35.852828: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-28 22:39:35.890214: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (224, 224))
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

full_images, full_labels = load_images_from_folder('Full', 0)
half_images, half_labels = load_images_from_folder('Half', 1)
low_images, low_labels = load_images_from_folder('Low', 2)
no_images, no_labels= load_images_from_folder("Absent", 3)

images = np.concatenate((full_images, half_images, low_images, no_images))
labels = np.concatenate((full_labels, half_labels, low_labels, no_labels))
images = images / 255.0

In [3]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

In [4]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

with tf.device("/GPU:0"):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

I0000 00:00:1724864997.686805    5620 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724864997.775069    5620 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724864997.775165    5620 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724864997.779662    5620 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724864997.779738    5620 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 128)               65664     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 516       
                                                                 
Total params: 14780868 (56.38 MB)
Trainable params: 66180 (258.52 KB)
Non-trainable params: 14714688 (56.13 MB)
__________

In [5]:
with tf.device("/GPU:0"):
    history= model.fit(
        datagen.flow(X_train, y_train, batch_size= 32),
        epochs= 5, 
        validation_data= (X_test, y_test)
    )

Epoch 1/5


2024-08-28 22:40:00.865145: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902
W0000 00:00:1724865001.035555   32863 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865001.203997   32863 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865001.207428   32863 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865001.210834   32863 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865001.215209   32863 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865001.219543   32863 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865001.235176   32863 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865001.240599   32863 gpu_t

13/15 [=========================>....] - ETA: 0s - loss: 1.5355 - accuracy: 0.2692

W0000 00:00:1724865007.159428   32862 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865007.162848   32862 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865007.166442   32862 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865007.169406   32862 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865007.172649   32862 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865007.176622   32862 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865007.180049   32862 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865007.184892   32862 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865007.188283   32862 gp

15/15 [==============================] - ETA: 0s - loss: 1.5256 - accuracy: 0.2777

W0000 00:00:1724865009.502032   32859 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865009.505315   32859 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865009.508182   32859 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865009.511006   32859 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865009.514174   32859 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865009.518267   32859 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865009.521797   32859 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865009.525930   32859 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865009.529591   32859 gp

15/15 [==============================] - 11s 445ms/step - loss: 1.5256 - accuracy: 0.2777 - val_loss: 1.2261 - val_accuracy: 0.7333
Epoch 2/5


W0000 00:00:1724865010.552192   32859 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865010.565560   32859 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865010.569509   32859 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865010.573503   32859 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865010.577501   32859 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865010.580736   32859 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865010.584963   32859 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865010.589019   32859 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865010.593110   32859 gp

15/15 [==============================] - 4s 277ms/step - loss: 1.2468 - accuracy: 0.4551 - val_loss: 1.0783 - val_accuracy: 0.9667
Epoch 3/5
15/15 [==============================] - 4s 284ms/step - loss: 1.0875 - accuracy: 0.6117 - val_loss: 0.9290 - val_accuracy: 0.7917
Epoch 4/5
15/15 [==============================] - 4s 284ms/step - loss: 0.9348 - accuracy: 0.7620 - val_loss: 0.8081 - val_accuracy: 0.8417
Epoch 5/5
15/15 [==============================] - 4s 285ms/step - loss: 0.8114 - accuracy: 0.8330 - val_loss: 0.6779 - val_accuracy: 0.9583


In [6]:
model.save("H5/vgg16.h5")

/home/manankhatri/.local/lib/python3.10/site-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

4/4 [==============================] - 1s 123ms/step - loss: 0.6779 - accuracy: 0.9583
Test Accuracy: 95.83%
